# Imports

In [2]:
import pandas as pd
import numpy as np

# Constants

In [65]:
TRAIN_DATA_FILE = '02_Training/train-data.txt'
TEST_DATA_FILE = '02_Training/test-data.txt'
TOKEN_SPAM_PROB_FILE = '03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = '03_Testing/prob-ham.txt'
TOKEN_ALL_PROB_FILE = '03_Testing/prob-all-tokens.txt'
TEST_TARGET_FILE = '03_Testing/test_target.txt'
TEST_FEATURE_MATRIX = '03_Testing/test_features.txt'

VOCAB_SIZE = 2500

In [6]:
sparse_train_data = np.loadtxt(TRAIN_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [18]:
sparse_train_data = np.delete(sparse_train_data, 0, 1)

In [136]:
sparse_train_data.shape

(258365, 4)

In [137]:
sparse_test_data.shape

(117791, 4)

In [7]:
#Create empty dataframe
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names

['DOC_ID',
 'CATEGORY',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,


In [141]:
index_names = np.unique(sparse_test_data[:,0])
index_names

array([   8,   12,   14, ..., 5788, 5792, 5793])

In [11]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [140]:
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    4],
       [   8,    4,    1,    2],
       ...,
       [5793, 2377,    0,    4],
       [5793, 2399,    0,    3],
       [5793, 2468,    0,    1]])

In [12]:
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Full matrix from a sparse matrix

In [142]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, nr_words))
    doc_id_names = np.unique(sparse_matrix[:,0])
    
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        category = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID']=doc_nr
        full_matrix.at[doc_nr, 'CATEGORY']=category
        full_matrix.at[doc_nr, word_id]=occurence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    full_matrix.fillna(value=0, inplace=True)
    
    return full_matrix

In [143]:
%%time
full_train_data=make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 41.5 s


In [144]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [145]:
full_train_data.shape

(4013, 2501)

## Training the model

### Probability of the email being spam

In [146]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
prob_spam

0.310989284824321

## Totl number of words/tokens

In [147]:
full_train_feature = full_train_data.loc[:, full_train_data.columns!='CATEGORY']
full_train_feature

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
email_lengths = full_train_feature.sum(axis=1)
email_lengths.shape

(4013,)

In [149]:
total_wc = email_lengths.sum()
total_wc


429231

## Nummber of tokens in spam and ham

In [150]:
spam_length=email_lengths[full_train_data.CATEGORY == 1]
spam_length.shape

(1248,)

In [151]:
spam_wc = spam_length.sum()
spam_wc

176348

In [152]:
ham_length=email_lengths[full_train_data.CATEGORY == 0]
ham_length.shape

(2765,)

In [153]:
ham_wc = ham_length.sum()
ham_wc

252883

In [154]:
total_wc-spam_wc-ham_wc

0

In [155]:
print('Average number of words in spam email: ', spam_wc/spam_length.shape[0])
print('Average number of words in spam email: ', ham_wc/ham_length.shape[0])

Average number of words in spam email:  141.30448717948718
Average number of words in spam email:  91.45858951175407


## Summing the tokens occuring in spam

In [156]:
full_train_feature.shape

(4013, 2500)

In [157]:
train_spam_tokens = full_train_feature.loc[full_train_data.CATEGORY==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
train_spam_tokens.shape

(1248, 2500)

In [159]:
summed_spam_tokens = train_spam_tokens.sum(axis=0)+1
summed_spam_tokens

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495       2
2496       8
2497      12
2498      21
2499      10
Length: 2500, dtype: int64

## Summing the tokens occuring in ham

In [160]:
train_ham_tokens = full_train_feature.loc[full_train_data.CATEGORY==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1
summed_ham_tokens

0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495      22
2496      24
2497      16
2498      10
2499      22
Length: 2500, dtype: int64

## P(Token|Spam): Probability that a token occurs given that it is spam

In [161]:
prob_spam_tokens = summed_spam_tokens/(spam_wc + VOCAB_SIZE)
prob_spam_tokens

0       0.012184
1       0.005228
2       0.006805
3       0.011306
4       0.006816
          ...   
2495    0.000011
2496    0.000045
2497    0.000067
2498    0.000117
2499    0.000056
Length: 2500, dtype: float64

In [162]:
prob_spam_tokens.sum()

1.0

## P(Token|ham): Probability that a token occurs given that it is ham

In [163]:
prob_ham_tokens = summed_ham_tokens/(ham_wc + VOCAB_SIZE)
prob_ham_tokens

0       0.021474
1       0.010142
2       0.008008
3       0.003673
4       0.006312
          ...   
2495    0.000086
2496    0.000094
2497    0.000063
2498    0.000039
2499    0.000086
Length: 2500, dtype: float64

In [164]:
prob_ham_tokens.sum()

1.0

## P(Token): Probability that a token occurs

In [165]:
prob_tokens_all = full_train_feature.sum(axis=0)/total_wc

In [166]:
prob_tokens_all.sum()

1.0

In [167]:
#saving data
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_spam_tokens)

In [168]:
np.savetxt(TOKEN_HAM_PROB_FILE, prob_ham_tokens)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Preparing test data

In [169]:
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    4],
       [   8,    4,    1,    2],
       ...,
       [5793, 2377,    0,    4],
       [5793, 2399,    0,    3],
       [5793, 2468,    0,    1]])

In [170]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 18 s


In [174]:
full_test_data.shape

(1724, 2501)

In [172]:
X_test = full_test_data.loc[:, full_test_data.columns!='CATEGORY']
y_test = full_test_data.CATEGORY
X_test

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,2,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,5,5,2,2,1,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,4,0,2,4,3,3,1,4,3,...,0,0,0,0,0,0,0,0,0,0


In [173]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)